<a href="https://colab.research.google.com/github/Staffilon/KestraDataOrchestrator/blob/main/Analysis_and_Exploration/pieces_produced_dataset_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd "gdrive/MyDrive/Dati-TBDM/energy_bak/energy_bak"

/content/gdrive/.shortcut-targets-by-id/1xGTanGyuk-TCLCMoGz5VNG7sUEscVC-Z/Dati-TBDM/energy_bak/energy_bak


In [3]:
import pandas as pd
import os
from multiprocessing import Pool, cpu_count

In [4]:
# Define the working directory
working_directory = os.getcwd()

Controls how many files fail to open. Initially, 90 files couldn't open because of messed up columns

In [ ]:
def process_file(filename):
    file_path = os.path.join(folder_path, filename)
    try:
        pd.read_csv(file_path, encoding='latin1')
    except pd.errors.ParserError:
        print(f"Failed to open file: {filename}")
        return 1
    return 0

if __name__ == '__main__':
    folder_path = os.getcwd()  # Get the current working directory
    error_count = 0

    file_list = [filename for filename in os.listdir(folder_path) if filename.endswith(".csv")]

    with Pool() as pool:
        error_count += sum(pool.map(process_file, file_list))

    print(f"Total number of files with parsing errors: {error_count}")


# File Formatting
Files have columns with shifted values, making them impossible to open with pandas.
In this section we will format them

Fix the colulmns of the files. Columns could iether:

*   Be shifted by one field and have a 5,a 9 or a 51/2" on column 3/4
*   Be shifted by one field and have a 0 on column 8

In [ ]:
# Function to preprocess CSV files
def preprocess_csv_files(folder_path):
    # Get a list of all files in the directory
    file_list = os.listdir(folder_path)

    # Iterate over each file in the directory
    for filename in file_list:
        # Check if the file starts with "Tormatic" and ends with ".csv"
        if filename.startswith("Tormatic") and filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            print("Processing file:", filename)

            # Process the CSV file
            with open(file_path, 'r', encoding='latin1') as file:
                lines = file.readlines()

            processed_lines = []
            for line in lines:
                fields = line.replace(' ', '').strip().split(',')

                # Check if the extra value '5' is present in the third column
                if len(fields) >= 3 and fields[2] == '5':
                    # If '5' is present in the third column, remove it
                    fields.pop(2)
                elif len(fields) >= 4 and fields[3] == '5':
                    # If '5' is present in the fourth column, remove it
                    fields.pop(3)
                elif len(fields) == 8 and fields[7] == '0':
                    # If '5' is present in the fourth column, remove it
                    fields.pop(7)
                    print("Deleted value '0'")

                # Reconstruct the line without the removed column
                line = ','.join(fields) + '\n'  # Assuming comma-separated values
                processed_lines.append(line)

            # Write the processed lines back to the file
            with open(file_path, 'w') as file:
                file.writelines(processed_lines)

            print("File", filename, "processed and saved.")

# Preprocess all CSV files in the working directory
preprocess_csv_files(working_directory)

In [ ]:
# Define the path to the CSV file
file_path = 'Tormatic_20230531_ 3303207.csv'

# Function to preprocess the CSV file
def preprocess_csv(file_path):
    # Read the CSV file into a list of lines
    with open(file_path, 'r', encoding='latin1') as file:
        lines = file.readlines()

    # Replace or remove the problematic value from each line
    processed_lines = []
    for line in lines:
        # Check if the problematic value is present in the line
        if "51/2" in line:
            # Replace "5 1/2" with an empty string to remove it
            line = line.replace("51/2", "")
            print('deleted')
        processed_lines.append(line)

    # Write the processed lines back to the CSV file
    with open(file_path, 'w', encoding='latin1') as file:
        file.writelines(processed_lines)

# Preprocess the CSV file
preprocess_csv(file_path)

deleted


In [ ]:
# Define the path to the CSV file
file_path = 'Tormatic_20230530_12522729.csv'

# Process the CSV file
with open(file_path, 'r', encoding='latin1') as file:
    lines = file.readlines()

processed_lines = []
for line in lines:
    fields = line.replace(' ', '').strip().split(',')

    # Check if the extra value '9' is present in the third column
    if len(fields) >= 3 and fields[2] == '9':
        # If '9' is present in the third column, remove it
        fields.pop(2)
        print("deleted on third column")
    elif len(fields) >= 4 and fields[3] == '9':
        # If '9' is present in the fourth column, remove it
        fields.pop(3)
        print("deleted on fourth column")
    elif len(fields) >= 4 and fields[3] == '51/2"':
        # If '51/2"' is present in the fourth column, remove it
        fields.pop(3)
        print("deleted 51/2 on fourth column")


    # Reconstruct the line without the removed column
    line = ','.join(fields) + '\n'  # Assuming comma-separated values
    processed_lines.append(line)

# Write the processed lines back to the file
with open(file_path, 'w') as file:
    file.writelines(processed_lines)

print("File", file_path, "processed and saved.")

# Deletion of unusable files
There are a couple of files that have no use and cannot be merged in the final dataset

In [ ]:
df = pd.read_csv('Tormatic_20211217_18331426.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID_ROW      4 non-null      object
 1   COD_TORNIO  3 non-null      object
 2   COD_PART    3 non-null      object
 3   TS_START    3 non-null      object
 4   TS_END      3 non-null      object
 5   N_PTZ       3 non-null      object
 6   EXP_STATUS  3 non-null      object
dtypes: object(7)
memory usage: 352.0+ bytes


In [ ]:
df = pd.read_csv('Tormatic_20211217_18312033.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID_ROW      3 non-null      object
 1   COD_TORNIO  2 non-null      object
 2   COD_PART    2 non-null      object
 3   TS_START    2 non-null      object
 4   TS_END      2 non-null      object
 5   N_PTZ       2 non-null      object
 6   EXP_STATUS  2 non-null      object
dtypes: object(7)
memory usage: 296.0+ bytes


By comparison these are the columns needed

In [ ]:
df = pd.read_csv('Tormatic_20231219_ 3303217.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20855 entries, 0 to 20854
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ID                 20855 non-null  object
 1   COD_MACC           20854 non-null  object
 2   COD_ART            20854 non-null  object
 3   TIMESTAMP_INIZIO   20854 non-null  object
 4   TIMESTAMP_FINE     20854 non-null  object
 5   NUMERO_PEZZI_PROD  20854 non-null  object
 6   EXP_STATUS         20854 non-null  object
 7   ODP                20854 non-null  object
dtypes: object(8)
memory usage: 1.3+ MB


There are also files couldn't be filled because of some errors.

In [ ]:
df = pd.read_csv('Tormatic_20220702_ 3310157.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
 #   Column                                                                                                                Non-Null Count  Dtype 
---  ------                                                                                                                --------------  ----- 
 0   Sqlcmd:errore:MicrosoftODBCDriver17forSQLServer:ProviderSSL:Connessioneincorsointerrottaforzatamentedall'hostremoto.  2 non-null      object
dtypes: object(1)
memory usage: 144.0+ bytes


In [ ]:
df = pd.read_csv('Tormatic_20220715_ 3310169.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
 #   Column                                                                                                                Non-Null Count  Dtype 
---  ------                                                                                                                --------------  ----- 
 0   Sqlcmd:errore:MicrosoftODBCDriver17forSQLServer:ProviderSSL:Connessioneincorsointerrottaforzatamentedall'hostremoto.  2 non-null      object
dtypes: object(1)
memory usage: 144.0+ bytes


The simplest choice is to delete these files since their entries are insignificant

In [ ]:
# List of file names to delete
files_to_delete = ["Tormatic_20211217_18331426.csv",
                   "Tormatic_20211217_18312033.csv",
                   "Tormatic_20220702_ 3310157.csv",
                   "Tormatic_20220715_ 3310169.csv"]

# Iterate over the list of files and delete each one
for filename in files_to_delete:
    try:
        os.remove(filename)
        print(f"File '{filename}' deleted successfully.")
    except OSError as e:
        print(f"Error deleting file '{filename}': {e}")

# Dataset Cleaning
This section involves taking out useless data and columns.
First, I want to check if there are files that do not have the target column

In [ ]:
# Function to check if a file has the column 'NUMERO_PEZZI_PROD'
def has_column(file_path):
    try:
        df = pd.read_csv(file_path, encoding='latin1')
        return 'NUMERO_PEZZI_PROD' not in df.columns
    except Exception as e:
        print(f"Error reading file '{file_path}': {e}")
        return True  # Treat as if the column is not present

# Function to process each file and check if it has the column
def process_file(filename):
    file_path = os.path.join(working_directory, filename)
    if has_column(file_path):
        print(f"File '{filename}' does not have the column 'NUMERO_PEZZI_PROD'")

# Get a list of all files in the directory starting with "Tormatic"
file_list = [f for f in os.listdir(working_directory) if f.startswith("Tormatic") and f.endswith(".csv")]

# Create a Pool of worker processes
with Pool() as pool:
    # Map the process_file function to each file in the file list
    pool.map(process_file, file_list)

The below code will delete all entries with 0 produced pieces

In [ ]:
# Function to preprocess a single CSV file
def preprocess_csv_file(file_path):
    print("Processing file:", os.path.basename(file_path))

    # Read the CSV file into a DataFrame and filter rows where NUMERO_PEZZI_PROD is not 0
    df = pd.read_csv(file_path, encoding='latin1')
    df = df[df['NUMERO_PEZZI_PROD'] != 0]

    # Write the modified DataFrame back to the CSV file
    df.to_csv(file_path, index=False, encoding='latin1')

    print("File", os.path.basename(file_path), "processed and saved.")

# Function to preprocess CSV files using multiprocessing
def preprocess_csv_files_multiprocessing(folder_path):
    # Get a list of all files in the directory
    file_list = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.startswith("Tormatic") and f.endswith(".csv")]

    # Get the number of CPU cores
    num_cores = cpu_count()

    # Create a Pool of worker processes
    with Pool(num_cores) as pool:
        # Map the preprocess_csv_file function to each file in parallel
        pool.map(preprocess_csv_file, file_list)

# Preprocess all CSV files in the working directory using multiprocessing
preprocess_csv_files_multiprocessing(working_directory)

The next step involves deleting columns that are not useful:
- EXP_STATUS
- ODP


In [ ]:
# Function to drop specified columns from a DataFrame
def drop_columns(df):
    columns_to_drop = ['EXP_STATUS', 'ODP']
    return df.drop(columns=columns_to_drop, errors='ignore')

# Function to process each file and drop specified columns
def process_file(filename):
    file_path = os.path.join(working_directory, filename)
    try:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path, encoding='latin1')

        # Drop specified columns
        df = drop_columns(df)

        # Save the modified DataFrame back to the CSV file
        df.to_csv(file_path, index=False, encoding='latin1')

        print(f"Columns 'EXP_STATUS' and 'ODP' dropped from file '{filename}'")
    except Exception as e:
        print(f"Error processing file '{filename}': {e}")

# Get a list of all files in the directory starting with "Tormatic"
file_list = [f for f in os.listdir(working_directory) if f.startswith("Tormatic") and f.endswith(".csv")]

# Create a Pool of worker processes
with Pool() as pool:
    # Map the process_file function to each file in the file list
    pool.map(process_file, file_list)

# Dataset merging
The final step involves merging all the datasets info a final one

In [ ]:
%cd "/content/gdrive/MyDrive/Dati-TBDM/energy_bak/energy_bak"

In [ ]:
# Function to read a CSV file into a DataFrame
def read_csv_to_df(file_path):
    try:
        return pd.read_csv(file_path, encoding='latin1')
    except Exception as e:
        print(f"Error reading file '{file_path}': {e}")
        return None

# Define the path where the merged dataset will be saved
output_folder = '/content/gdrive/MyDrive/Dati-TBDM/refined_data'
output_file = os.path.join(output_folder, 'produced_pieces_dataset.csv')

# Get a list of all files in the directory starting with "Tormatic"
file_list = [f for f in os.listdir(working_directory) if f.startswith("Tormatic") and f.endswith(".csv")]

# Create a Pool of worker processes
with Pool() as pool:
    # Map the read_csv_to_df function to each file in the file list
    dfs = pool.map(read_csv_to_df, [os.path.join(working_directory, filename) for filename in file_list])

# Filter out None values (indicating errors during reading)
dfs = [df for df in dfs if df is not None]

# Concatenate all DataFrames into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

# Save the merged DataFrame to a CSV file
try:
    os.makedirs(output_folder, exist_ok=True)  # Create the output folder if it doesn't exist
    merged_df.to_csv(output_file, index=False)
    print(f"Merged dataset saved to '{output_file}'")
except Exception as e:
    print(f"Error saving merged dataset: {e}")

# Dataset exploration

In [22]:
%cd "/content/gdrive/MyDrive/Dati-TBDM/refined_data"

/content/gdrive/.shortcut-targets-by-id/1xGTanGyuk-TCLCMoGz5VNG7sUEscVC-Z/Dati-TBDM/refined_data


In [75]:
final_df = pd.read_csv('produced_pieces_dataset.csv')

Checking if the values of the various columns have mixed values

In [43]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7552500 entries, 3 to 7566030
Data columns (total 6 columns):
 #   Column             Dtype  
---  ------             -----  
 0   ID                 object 
 1   COD_MACC           object 
 2   COD_ART            object 
 3   TIMESTAMP_INIZIO   object 
 4   TIMESTAMP_FINE     object 
 5   NUMERO_PEZZI_PROD  float64
dtypes: float64(1), object(5)
memory usage: 403.3+ MB


In [11]:
final_df.describe()

,ID,COD_MACC,COD_ART,TIMESTAMP_INIZIO,TIMESTAMP_FINE,NUMERO_PEZZI_PROD
count,7566032,7565739,7565333,7565738,7565738,7565738
unique,7480285,281,1632,398676,398145,18127
top,--------------------,0203,0001700AVPWA04FDORFS04Y,5X10X20,2022-06-1609:00:00.000,0.0
freq,294,250282,182235,12488,583,5778044


In [10]:
final_df.head()

,ID,COD_MACC,COD_ART,TIMESTAMP_INIZIO,TIMESTAMP_FINE,NUMERO_PEZZI_PROD
0,--------------------,---------------,------------------------------,-----------------------,-----------------------,------------------------
1,101481,0101,1120400AVPTS.98325511-03,2022-05-3006:06:00.000,2022-05-3006:07:00.000,0.0
2,101482,0101,1120400AVPTS.98325511-03,2022-05-3006:07:00.000,2022-05-3006:09:00.000,0.0
3,101483,0101,1120400AVPTS.98325511-03,2022-05-3006:09:00.000,2022-05-3006:10:00.000,0.0
4,101484,0101,1120400AVPTS.98325511-03,2022-05-3006:10:00.000,2022-05-3006:12:00.000,0.0


The below columns have mixed data which has to be deleted, like dates where integers are expected

In [74]:
for col in ['ID', 'COD_MACC', 'NUMERO_PEZZI_PROD']:
    print(f"Unique values in '{col}':")
    print(final_df[col].value_counts())
    print()

Unique values in 'ID':
51799       2
74998       2
74996       2
74995       2
74994       2
           ..
3591894     1
3591893     1
3591892     1
3591891     1
10837113    1
Name: ID, Length: 7467151, dtype: int64

Unique values in 'COD_MACC':
203.0     280373
202.0     277259
210.0     263987
209.0     260936
109.0     254664
204.0     246098
101.0     232726
207.0     232454
208.0     228778
205.0     219338
201.0     218861
214.0     191130
104.0     182946
103.0     182088
211.0     177952
105.0     173145
213.0     170510
106.0     166584
107.0     159555
212.0     154146
602.0     117578
604.0     115953
609.0     103778
318.0     103751
601.0     101179
102.0     100010
603.0      96043
319.0      95975
110.0      94415
613.0      94098
614.0      91168
607.0      90263
605.0      89997
610.0      87005
606.0      85301
303.0      84309
313.0      82267
302.0      81964
111.0      79629
616.0      79602
515.0      78049
301.0      77451
311.0      76979
304.0      75271
215.0

# Removing data from the number of pieces created that is in date format

In [ ]:
# Convert all values in the column to strings
final_df['NUMERO_PEZZI_PROD'] = final_df['NUMERO_PEZZI_PROD'].astype(str)

# Filter out rows with strings longer than 10 characters or length 0
filtered_df = final_df[(final_df['NUMERO_PEZZI_PROD'].str.len() <= 10) & (final_df['NUMERO_PEZZI_PROD'].str.len() > 0)]

# Print the number of rows removed
print("Number of rows removed:", len(final_df) - len(filtered_df))

In [60]:
filtered_df.to_csv("produced_pieces_dataset.csv", index=False)

# Removing text from the id column and making it universal

In [61]:
final_df = pd.read_csv('produced_pieces_dataset.csv')

In [ ]:
for col in ['ID']:
    print(f"Unique values in '{col}':")
    print(final_df[col].value_counts())
    print()

Check if there are strings inside

In [76]:
# Attempt to convert the values in the 'ID' column to numeric (integer)
try:
    pd.to_numeric(final_df['ID'], errors='raise', downcast='integer')
    print("The 'ID' column contains only integer values.")
except ValueError:
    print("The 'ID' column contains non-integer values.")

The 'ID' column contains only integer values.


Make all the ids unique

In [77]:
# Reset the index to ensure the DataFrame is properly indexed
final_df.reset_index(drop=True, inplace=True)

# Assign incremental IDs starting from 1
final_df['ID'] = final_df.reset_index().index + 1

Check if there are duplicates

In [78]:
# Check for duplicate IDs
duplicates = final_df['ID'].duplicated().any()

# Print the result
if duplicates:
    print("Duplicate IDs found!")
else:
    print("All IDs are unique.")

All IDs are unique.


Save the dataset

In [80]:
final_df.to_csv("produced_pieces_dataset.csv", index=False)

#Final dataset analysis

In [81]:
final_df = pd.read_csv('produced_pieces_dataset.csv')

Check if there are empty values and deleting them if that is the case

In [99]:
# Check for missing values in any row
missing_values = final_df.isnull().any(axis=1).any()

if missing_values:
    print("Dataset has missing values in some rows.")
else:
    print("Dataset has no missing values in any row.")

Dataset has no missing values in any row.


Drop the rows with missing values

In [100]:
# Drop rows with missing values
final_df = final_df.dropna()

# Confirm that missing rows are removed
print("Rows with missing values have been removed.")

Rows with missing values have been removed.


Modify the values of dates that are not following the format

In [ ]:
# Preprocess the TIMESTAMP_INIZIO column to add a space between date and time
final_df['TIMESTAMP_INIZIO'] = final_df['TIMESTAMP_INIZIO'].str.replace(r'(\d{4}-\d{2}-\d{2})(\d{2}:\d{2}:\d{2}\.\d{3})', r'\1 \2')
final_df['TIMESTAMP_FINE'] = final_df['TIMESTAMP_FINE'].str.replace(r'(\d{4}-\d{2}-\d{2})(\d{2}:\d{2}:\d{2}\.\d{3})', r'\1 \2')

Convert them to datetime in the dataset

Convert the columns of start and end timestamps to date

In [98]:
# Convert 'TIMESTAMP_INIZIO' and 'TIMESTAMP_FINE' columns to datetime
final_df['TIMESTAMP_INIZIO'] = pd.to_datetime(final_df['TIMESTAMP_INIZIO'])
final_df['TIMESTAMP_FINE'] = pd.to_datetime(final_df['TIMESTAMP_FINE'])

# Confirm the conversion
print("Columns converted to datetime format:")
print(final_df.dtypes)

Columns converted to datetime format:
ID                            int64
COD_MACC                    float64
COD_ART                      object
TIMESTAMP_INIZIO     datetime64[ns]
TIMESTAMP_FINE       datetime64[ns]
NUMERO_PEZZI_PROD           float64
dtype: object


Convert COD_ART to string

In [103]:
final_df['COD_ART'] = final_df['COD_ART'].astype(str)

# Confirm the conversion
print("Columns converted to datetime format:")
print(final_df.dtypes)

Columns converted to datetime format:
ID                            int64
COD_MACC                    float64
COD_ART                      object
TIMESTAMP_INIZIO     datetime64[ns]
TIMESTAMP_FINE       datetime64[ns]
NUMERO_PEZZI_PROD           float64
dtype: object


Save the dataset

In [104]:
final_df.to_csv("produced_pieces_dataset.csv", index=False)